In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
901,**** WARNING: here be spoilers **** Why do I w...,negative
586,"First of all, no one with any law enforcement ...",negative
303,Soul Calibur is more solid than it ever was......,positive
164,"In terms of the arts, the 1970s were a very tu...",positive
727,What is supposed to be a simple generic myster...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\krush\AppData\Local\Temp\ipykernel_4996\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
901,warning spoiler waste hastily fleeing year wat...,negative
586,first all one law enforcement experience not e...,negative
303,soul calibur solid ever wa new character creat...,positive
164,term art s turbulent era literature visual art...,positive
727,supposed simple generic mystery plot involving...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    272
positive    228
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
901,warning spoiler waste hastily fleeing year wat...,0
586,first all one law enforcement experience not e...,0
303,soul calibur solid ever wa new character creat...,1
164,term art s turbulent era literature visual art...,1
727,supposed simple generic mystery plot involving...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/krushnakant-borude/End_to_End_MLOPs_Capstone_Project.mlflow')
dagshub.init(repo_owner='krushnakant-borude', repo_name='End_to_End_MLOPs_Capstone_Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as krushnakant-borude

Initialized MLflow to track repo "krushnakant-borude/End_to_End_MLOPs_Capstone_Project"

Repository krushnakant-borude/End_to_End_MLOPs_Capstone_Project initialized!

2025/12/30 16:49:32 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e26ec7b9222348a29c2b59116b761b73', creation_time=1767093573176, experiment_id='0', last_update_time=1767093573176, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.23)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-30 16:54:35,913 - INFO - Starting MLflow run...
2025-12-30 16:54:36,290 - INFO - Logging preprocessing parameters...
2025-12-30 16:54:37,434 - INFO - Initializing Logistic Regression model...
2025-12-30 16:54:37,435 - INFO - Fitting the model...
2025-12-30 16:54:37,476 - INFO - Model training complete.
2025-12-30 16:54:37,476 - INFO - Logging model parameters...
2025-12-30 16:54:37,893 - INFO - Making predictions...
2025-12-30 16:54:37,893 - INFO - Calculating evaluation metrics...
2025-12-30 16:54:37,906 - INFO - Logging evaluation metrics...
2025-12-30 16:54:39,437 - INFO - Saving and logging the model...
2025/12/30 16:54:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-30 16:54:52,960 - INFO - Model training and logging completed in 16.67 seconds.
2025-12-30 16:54:52,961 - INFO - Accuracy: 0.6869565217391305
2025-12-30 16:54:52,962 - INFO - Precision: 0.675
2025-12-30 16:54:52,963 - INFO - Recall: 0.54
2025-12-30 16:54:52,963 

🏃 View run tasteful-turtle-695 at: https://dagshub.com/krushnakant-borude/End_to_End_MLOPs_Capstone_Project.mlflow/#/experiments/0/runs/1110869c8021499896b6322765dc3fa3
🧪 View experiment at: https://dagshub.com/krushnakant-borude/End_to_End_MLOPs_Capstone_Project.mlflow/#/experiments/0
